In [1]:
%pylab --no-import-all inline

Populating the interactive namespace from numpy and matplotlib


# Most general form of cross-validation, with history
---

This provides little personalization, and still avoids the issue of using a subject's future data for prediction.

In [2]:
from os import path
import sys

import pandas as pd
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

# add the 'src' directory as one where we can import modules
src_dir = path.join("..", 'src')
sys.path.append(src_dir)

# import my method from the source code
%aimport features.build_features
%aimport models.fit_predict
%aimport visualization.visualize
from features.build_features import previous_value
from models.fit_predict import cv_predict
from visualization.visualize import modified_bland_altman_plot

In [3]:
file = path.join("..", "data", "interim", "df.csv")
df = pd.read_csv(file, index_col=0)

## Compute features

In [4]:
features = []

### Previous `L_PREOVULATION` and `L_CYCLE`

In [5]:
df['past_L_PREOVULATION'] = previous_value('L_PREOVULATION', df)
df['past_L_CYCLE'] = previous_value('L_CYCLE', df)

df.dropna(subset=[
    'past_L_PREOVULATION', 
    'past_L_CYCLE'
], inplace=True)

features += ['past_L_PREOVULATION', 'past_L_CYCLE']

### $n$ days of temperature measurements.

The use case requires deleting those whose ovulation occurs before these $n$ days.

In [6]:
NUMBER_OF_DAYS = 10
df = df[df.L_PREOVULATION > NUMBER_OF_DAYS]  # No use predicting backward in time.
temp_measurements = ["TEMP" + str(i + 1) for i in range(NUMBER_OF_DAYS)]
features += temp_measurements

In [7]:
features

['past_L_PREOVULATION',
 'past_L_CYCLE',
 'TEMP1',
 'TEMP2',
 'TEMP3',
 'TEMP4',
 'TEMP5',
 'TEMP6',
 'TEMP7',
 'TEMP8',
 'TEMP9',
 'TEMP10']

In [8]:
WINDOW_SIZE = 10

TEMP_COLUMNS = ["TEMP" + chr(i + ord("A")) for i in range(WINDOW_SIZE)]
DISTANCE_COLUMN = ["DISTANCE"]
OTHER_COLUMNS = ["COUNTDOWN", "past_L_PREOVULATION", "past_L_CYCLE", "ID"]
STARTING_COLUMNS = TEMP_COLUMNS + DISTANCE_COLUMN + OTHER_COLUMNS
df2 = pd.DataFrame(columns=STARTING_COLUMNS)

for i in range(99 - WINDOW_SIZE):
    df['COUNTDOWN'] = df.L_PREOVULATION - (i + WINDOW_SIZE + 1)
    df['DISTANCE'] = i
    columns = ["TEMP" + str(i + j + 1) for j in range(WINDOW_SIZE)]
    df2.columns = columns + DISTANCE_COLUMN + OTHER_COLUMNS
    df2 = df2.append(df[df.COUNTDOWN > 0][columns + DISTANCE_COLUMN + OTHER_COLUMNS], ignore_index=True)
df2.columns = STARTING_COLUMNS
df2.dropna(subset=TEMP_COLUMNS, thresh=WINDOW_SIZE - 1, inplace=True)

In [9]:
X = df2.drop(labels=['COUNTDOWN', 'ID'], axis=1)
y = df2.COUNTDOWN
grouping = df2.ID

In [10]:
X.columns

Index(['TEMPA', 'TEMPB', 'TEMPC', 'TEMPD', 'TEMPE', 'TEMPF', 'TEMPG', 'TEMPH',
       'TEMPI', 'TEMPJ', 'DISTANCE', 'past_L_PREOVULATION', 'past_L_CYCLE'],
      dtype='object')

## Perform regression

In [ ]:
from sklearn.model_selection import cross_val_predict, GroupKFold
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
mlpr = MLPRegressor(random_state=1337, hidden_layer_sizes=(50, 20))
imp = Imputer(strategy='mean')
scl = StandardScaler()
pipeline = Pipeline([('imp', imp), ('scl', scl), ('mlp', mlpr)])

cv = GroupKFold(n_splits=10)

y_pred = cross_val_predict(pipeline, X, y,
                           cv=cv, groups=grouping,
                           verbose=3, n_jobs=-1)

[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  3.0min remaining:  1.3min


In [ ]:
mean_squared_error(y_pred=y_pred, y_true=y)

In [ ]:
mean_absolute_error(y_pred=y_pred, y_true=y)

In [ ]:
modified_bland_altman_plot(y_pred, y);

## Discussion
---

Our features are only the first ten temperatures of the cycle and the participant's last cycle length and follicular phase length. With it, we achieve a MSE of about 12, which beats the Bortot paper's 15. In terms of use case, this is about equal to the Bortot result.

In [ ]:
df.L_PERIOD.median()

Now, the median period length is 5, which means that we are really using measurements of BBT during the period to determine the day of ovulation.

Since this model has only slight personalization, it's exciting to see how well a personalized model will do.